In [ ]:
m!gdown --id 1WtSZaXfOOm4cwetT3totzWWjHmZwsLt3
!gdown --id 1wnZtSwMZwvgd5Jxt4TfCeuIBI6CIQRCK
!pip install monpa
!pip install flashtext


Downloading...
From: https://drive.google.com/uc?id=1WtSZaXfOOm4cwetT3totzWWjHmZwsLt3
To: /content/tagging_article_sample.csv
100% 51.3k/51.3k [00:00<00:00, 19.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wnZtSwMZwvgd5Jxt4TfCeuIBI6CIQRCK
To: /content/stopwords.txt
100% 8.12k/8.12k [00:00<00:00, 14.2MB/s]
     |████████████████████████████████| 8.5MB 3.4MB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=3e641b085a6d34d993048dd5cf55308885e2d263cae284b4a83099ff7ca88661
  Stored in directory: /root/.cache/pip/wheels/37/db/d7/fe74f7cb8e5c3afed90fe6f4967c933a6f13d81ab6b3d3128c
Successfully built flashtext


# **Step 1. Preparing Dataset**

### 先了解資料吧！
* 利用 `pandas` 載入 `data.csv` 並且show 出資料格式

In [ ]:
import pandas as pd 

df = pd.read_csv("tagging_article_sample.csv")

### Cleaning Data
* 建立一個 `remove_url` Function
* 建立一個 `remove_emojis_punctuation` Function
* Input: `data_dir` (path//to//data)
* Output: DataFrame

In [ ]:
import pandas as pd
import re

def remove_url(df, column_name):
    df[column_name] = df[column_name].str.replace(
        r'https?://[^\s<>"]+|www\.[^\s<>"]+', " ")
    return df


def remove_emojis_punctuation(df, column_name):
    df[column_name] = df[column_name].str.replace(
        '[^\w\s]', " ")
    return df
    

def convert_full(df, column_name):
    def _convert_full(text):
        FULL2HALF = dict((i + 0xFEE0, i) for i in range(0x21, 0x7F))
        FULL2HALF[0x3000] = 0x20
        return str(text).translate(FULL2HALF)
    df[column_name] = df[column_name].map(
        _convert_full)
    return df


In [ ]:
df = remove_url(df, 'content')
df = remove_emojis_punctuation(df, 'content')
final = convert_full(df, 'content')

### **Tokenization**

In [ ]:

class WordSegmentation:

  def __init__(self):
    self._build()


  def cut(self,sentence):
    assert "Need to be implemented"


  def df_cut(self, df, column_name):
    result_cut = []
    for data in df[column_name].tolist():
      seg_list = self.cut(data)
      result_cut.append(seg_list)
    return result_cut


  def _build(self):
    assert "Need to be implemented"


class MonpaCutWord(WordSegmentation):
  import monpa
  def __init__(self, **kargs):
    super(MonpaCutWord,self).__init__()


  def cut(self, text, split_char='，'):
    result_cut = []
    for item in text.split(split_char):
        if item != "\n":
            result_cut.extend(self.monpa.cut(str(item+split_char)))
    return result_cut
  

  def load_userdict(self, usr_dict_path):
    self.monpa.load_userdict(usr_dict_path)


  def _build(self):
    self.monpa.use_gpu(True)


class JiebaCutWord(WordSegmentation):
  import jieba
  def __init__(self, dict_path=None,  usr_dict_path=None, usr_word_list=None):
    super(JiebaCutWord,self).__init__()
        

  def cut(self, text):
    result_cut = []
    seg_list = self.jieba.cut(text, cut_all=False)
    for seg in seg_list:
        result_cut.append(seg)
    return result_cut


  def load_userdict(self, usr_dict_path):
    self.jieba.load_userdict(usr_dict_path)


  def add_word(self, usr_word_list):
    for items in usr_word_list:
        self.jieba.add_word(items)


  def _build(self):
    self.jieba.re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&._ ]+)", re.U)

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [ ]:
cutword = JiebaCutWord()
cutword_result = cutword.df_cut(final, 'content')

Building prefix dict from the default dictionary ...
08/31/2020 09:03:57 - DEBUG - jieba - Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
08/31/2020 09:03:57 - DEBUG - jieba - Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.969 seconds.
08/31/2020 09:03:58 - DEBUG - jieba - Loading model cost 0.969 seconds.
Prefix dict has been built successfully.
08/31/2020 09:03:58 - DEBUG - jieba - Prefix dict has been built successfully.


In [ ]:
def readstopword(stopword_path):
  stop_word_path = stopword_path
  word_list = [line.strip() for line in open(
      stop_word_path, 'r', encoding='UTF-8').readlines()]
  stopwords_list = [x.lower() for x in word_list]
  return stopwords_list


def cut_list_clean(cutword_result, stopwords_list, df):
  stopword = set(stopwords_list)
  cutword_final = [[e for e in l if e not in stopword ] for l in cutword_result]
  df['tokenized'] = cutword_final
  df['tokenized'] = df.tokenized.apply(
  lambda x: ','.join([str(i) for i in x]))
  return df

stopwords_list = readstopword('stopwords.txt')
cutword_df = cut_list_clean(cutword_result, stopwords_list, final)

# **Step 2. Embedding**

In [ ]:
# model = Word2Vec(
#     sentences=corpus, size=300, min_count=1, workers=4, sg=1,
#     negative=5, iter=500, compute_loss=True, callbacks=[callback()])

In [ ]:
! gdown --id 1llSWKZ7viH6AglZtacJWGjXzKc_JFsux
! gdown --id 1jy7wt70oivIlX1KbNr6SRULGcoScodmR


Downloading...
From: https://drive.google.com/uc?id=1llSWKZ7viH6AglZtacJWGjXzKc_JFsux
To: /content/post_embedding.npy
4.26MB [00:00, 29.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jy7wt70oivIlX1KbNr6SRULGcoScodmR
To: /content/tag_embedding.npy
100% 301k/301k [00:00<00:00, 89.5MB/s]


In [ ]:
import numpy as np

post_embedding = np.load('post_embedding.npy', allow_pickle=True)
tag_embedding = np.load('tag_embedding.npy', allow_pickle=True)


# **Step 3. Tagging**

### Calculate Distance

In [ ]:
! gdown --id 1MgoJrvE_n5mKoihb0DBkL9H1fVdfQPu9

Downloading...
From: https://drive.google.com/uc?id=1MgoJrvE_n5mKoihb0DBkL9H1fVdfQPu9
To: /content/tag_dict.csv
100% 7.14k/7.14k [00:00<00:00, 12.5MB/s]


In [ ]:
import pandas as pd
tag_dict = pd.read_csv("tag_dict.csv")

In [ ]:
tag_dict

,tag,word,id
0,運動,運動,0
1,運動,健身房,1
2,運動,重訓,2
3,運動,有氧,3
4,運動,打球,4
...,...,...,...
371,社會議題,新冠肺炎,371
372,社會議題,covid19,372
373,社會議題,廢死,373
374,社會議題,總統大選,374


In [ ]:
def dot(v1, v2):
    return sum(map(operator.mul, v1, v2))

    
def consine(v1, v2, dis):
    sumyy = (v2**2).sum(1)
    sumxx = (v1**2).sum(1, keepdims=1)
    sumxy = v1.dot(v2.T)
    consin = (sumxy/np.sqrt(sumxx))/np.sqrt(sumyy)
    consin_new = [j for i in consin for j in i]
    max_distace = max(consin_new)

    if max_distace >= dis:
        max_label_id = np.argmax(consin_new, axis=0)
        return max_label_id, max_distace

cos_result = []
labels = np.array(tag_embedding)
for article in post_embedding:
    if len(article) == 0:
        cos_result.append([])
        continue
    map_ids = []
    for cluster_feat in article:
        cluster_feat = np.expand_dims(cluster_feat, 0)
        consine_result = consine(cluster_feat, labels, 0.65)
        map_ids.append(consine_result)
    cos_result.append(map_ids)


In [ ]:
def tagging_to_article(cos_result, df):
    for element in cos_result:
        element[:] = [x for x in element if x is not None]

    cos_result_update = []
    cos_final = []

    for i in range(len(cos_result)):
        data = sorted(cos_result[i], key=lambda x: x[-1])
        cos_result_update.append(data)

    for i in cos_result_update:
        final = [a[0] for a in i]
        final_update = list(final)
        cos_final.append(final_update)

    area_dict = dict(zip(tag_dict.id, tag_dict.word))
    mapping_result = []
    for i in cos_final:
        data = [area_dict.get(item, item) for item in i]
        mapping_result.append(data)
        
    df['consine'] = cos_result_update
    df['mapping_result'] = mapping_result
    df['mapping_result'] = df['mapping_result'].apply(
        lambda x: ' '.join(map(str, x)))
    return df


tagging_result = tagging_to_article(
    cos_result, cutword_df)

### Tag to Article

In [ ]:
import flashtext

def tokenized_update(df, tag_df):
  map_update = {}
  for i in tag_df["tag"].tolist():
      map_update[i] = tag_df[tag_df["tag"] == i]["word"].tolist()
  fl = flashtext.KeywordProcessor()
  fl.add_keywords_from_dict(map_update)
  df["mapping_tag"] = df["mapping_result"].apply(fl.replace_keywords)
  df['words_cnt'] = df.tokenized.apply(
        lambda x: len(str(x).split(',')))
  return df

mapping_result = tokenized_update(tagging_result, tag_dict)



from collections import Counter
mapping_result['final_tagging']  = ""
mapping_result.loc[(mapping_result.words_cnt > 1) & (mapping_result.words_cnt <= 15), ["final_tagging"]] = mapping_result["mapping_tag"].str.split().apply(lambda x: [k for k, v in Counter(x).items() if v>=1])
mapping_result.loc[(mapping_result.words_cnt > 15) & (mapping_result.words_cnt <= 50), ["final_tagging"]] = mapping_result["mapping_tag"].str.split().apply(lambda x: [k for k, v in Counter(x).items() if v>=2])
mapping_result.loc[(mapping_result.words_cnt > 50) & (mapping_result.words_cnt <= 80), ["final_tagging"]] = mapping_result["mapping_tag"].str.split().apply(lambda x: [k for k, v in Counter(x).items() if v>=3])
mapping_result.loc[(mapping_result.words_cnt > 80) & (mapping_result.words_cnt <= 150), ["final_tagging"]] = mapping_result["mapping_tag"].str.split().apply(lambda x: [k for k, v in Counter(x).items() if v>=4])
mapping_result.loc[mapping_result.words_cnt > 150, ["final_tagging"]] = mapping_result["mapping_tag"].str.split().apply(lambda x: [k for k, v in Counter(x).items() if v>=5])
# mapping_result = mapping_result[['platform', 'content', 'final_tagging']]
mapping_result

,platform,content,tokenized,consine,mapping_result,mapping_tag,words_cnt,final_tagging
0,fanpage,不僅桃花運接二連三 連財運都會超旺 虎妞編 \n \n \n \n 好的然後小編再...,"桃花,運接,二連, ,連財運,超旺, ,虎妞,編, ,\n, ,\n, ,\n, ,...","[(178, 0.65277445), (170, 0.6990977)]",開運 運勢,占卜 占卜,46,[占卜]
1,fanpage,真是好消息 國衛院投入的疫苗研發將進入動物實驗階段 小草西 國產疫苗進動物實驗階段 快篩...,"真是,好消息, ,國衛院,投入,疫苗,研發將,進入,動物,實驗,階段, ,小草,西, ,...","[(99, 0.7007473), (317, 1.0), (317, 1.0)]",新品 疫苗 疫苗,開箱 醫療 醫療,34,[醫療]
2,fanpage,三國系列遊戲真的是怎麼玩都玩不膩耶 \n \n 掌握手遊 電玩最熱資訊 快幫 遊戲6一波,"三國,系列,遊戲,真的,玩,玩,不膩,耶, ,\n, ,\n, ,掌握,手遊, ,電玩,...","[(49, 0.99999994), (47, 1.0), (51, 1.0), (47, ...",手遊 遊戲 電玩 遊戲,遊戲 遊戲 遊戲 遊戲,25,[遊戲]
3,fanpage,看郭董穿的背心就知道囉 \n \n 郭台銘 親民黨\n \n see translation,"郭董,穿,背心,知道,囉, ,\n, ,\n, ,郭台銘, ,親民黨,\n, ,\n, ...","[(110, 0.74993724), (109, 0.8446653)]",褲子 上衣,穿搭 穿搭,20,[穿搭]
4,fanpage,還沒打流感疫苗嗎 動作要快 公費疫苗快打完了 小草西 \n \n 吳鳳 小百合去露營囉...,"還沒,流感疫苗, ,動作, ,公費,疫苗,完, ,小草,西, ,\n, ,\n, ,吳...","[(317, 0.65709347), (119, 0.69556016), (119, 0...",疫苗 料理 料理 疫苗,醫療 美食 美食 醫療,49,"[醫療, 美食]"
5,fanpage,贈票活動 女鬼橋 真實校園撞鬼直播事件改編\n 紅衣小女孩 x 粽邪 製作團隊全新話題...,",贈票,活動, ,女鬼,橋, ,真實,校園,撞鬼,直播,事件,改編,\n, ,紅衣,小...","[(244, 0.6872942), (243, 0.7926514), (243, 0.9...",電視劇 電影 電影 校園,影視 影視 影視 校園,96,[]
6,fanpage,醫師提醒 開學日後若兒童出現呼吸道或發燒症狀 回家儘量不要接近爺爺奶奶 小草西 \n \n...,"醫師,提醒, ,開學,日後若,兒童,出現,呼吸道,發燒,症狀, ,回家,儘,量,接近,爺爺,...","[(318, 0.67242086), (69, 0.6944399), (371, 0.7...",小兒科 幼兒園 新冠肺炎 就醫 兒童,醫療 親子 社會議題 醫療 親子,70,[]
7,fanpage,真的好好聽喔 在你旁編 \n 她翻唱林宥嘉 兜圈 網讚 耳朵懷孕雙...,"真的,好好,聽,喔, ,旁編, ,\n, ,翻唱,林宥,嘉, ,兜圈...","[(299, 0.6894551), (296, 1.0)]",歌手 音樂,音樂 音樂,79,[]
8,fanpage,長時間戴口罩 很多人臉部肌膚都過敏了\n \n 戴口罩讓肌膚拉警報 理膚安心霜加神經胜肽 ...,"長,時間,戴,口罩, ,人,臉部,肌膚,過敏,\n, ,\n, ,戴,口罩,肌膚,拉,警報...","[(353, 0.6533826), (357, 0.6878303), (184, 0.7...",膠原蛋白 議題 肌膚 肌膚 敏感肌 財經 肌膚 肌膚,保健 社會議題 保養 保養 保養 財經 保養 保養,88,[保養]
9,fanpage,男生都追蹤化妝很濃的妹 可是都不要自己女友化濃妝 小草西 \n \n 男友愛看濃妝妹卻要...,"男生,追,蹤,化,妝,濃,妹, ,女友,化濃妝, ,小草,西, ,\n, ,\n, ,男...","[(357, 0.6878303), (31, 0.73594975), (218, 0.9...",議題 妝容 鬧大學 女友 男友 女友 鬧大學 財經,社會議題 彩妝 校園 感情 感情 感情 校園 財經,98,[]
